### A MNIST-like fashion product database

In this, we classify the images into respective classes given in the dataset. We use a Neural Net and a Deep Neural Net in Keras to solve this and check the accuracy scores.

### Load tensorflow

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import pandas as pd


TensorFlow 2.x selected.


In [3]:
tf.__version__

'2.1.0-rc1'

### Read the dataset in a new python notebook

In [4]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df_bank = pd.read_csv("/content/drive/My Drive/PGPAIML/30Nov/Assignment/Project-1/bank.csv")

In [6]:
df_bank.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Drop the columns which are unique for all users like IDs 

In [8]:
df_bank=df_bank.drop(['RowNumber','CustomerId','Surname'],axis=1)
df_bank.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:

from sklearn.preprocessing import LabelEncoder
# filter categorical columns using mask and turn it into a list
categorical_cols = df_bank.select_dtypes(exclude = ['int64','float64']).columns.tolist()

In [10]:
# instantiate labelencoder  
le = LabelEncoder()
# apply le on categorical feature columns
df_bank[categorical_cols] = df_bank[categorical_cols].apply(lambda col: le.fit_transform(col))
df_bank.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
5,645,2,1,44,8,113755.78,2,1,0,149756.71,1
6,822,0,1,50,7,0.00,2,1,1,10062.80,0
7,376,1,0,29,4,115046.74,4,1,0,119346.88,1
8,501,0,1,44,4,142051.07,2,0,1,74940.50,0
9,684,0,1,27,2,134603.88,1,1,1,71725.73,0


###Distinguish the feature and target set 

In [0]:
X=df_bank.drop('Exited',axis=1)
Y=df_bank['Exited']

###Divide the data set into Train and test sets

In [0]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

###Normalize the train and test data 

In [0]:
from sklearn import preprocessing
normalized_train_x = preprocessing.normalize(train_x)
normalized_test_x = preprocessing.normalize(test_x)

###Initialize &amp; build the model

In [15]:
normalized_train_x.shape

(7000, 10)

In [0]:
model2 = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(10,)),
                             tf.keras.layers.Dense(100,activation='sigmoid'),
                              tf.keras.layers.Dense(100,activation='sigmoid'),
                              tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

In [0]:
model2.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
batch_normalization (BatchNo (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 12,610
Trainable params: 12,410
Non-trainable params: 200
_________________________________________________________________


In [19]:
model2.fit(normalized_train_x, train_y, validation_data=(normalized_test_x, test_y), epochs=100,
          batch_size = normalized_train_x.shape[0])

Train on 7000 samples, validate on 3000 samples
Epoch 1/100
7000/7000 [==============================] - 1s 131us/sample - loss: 2.3388 - accuracy: 0.0000e+00 - val_loss: 2.2583 - val_accuracy: 0.0000e+00
Epoch 2/100
7000/7000 [==============================] - 0s 5us/sample - loss: 2.3209 - accuracy: 0.0000e+00 - val_loss: 2.2403 - val_accuracy: 0.0000e+00
Epoch 3/100
7000/7000 [==============================] - 0s 5us/sample - loss: 2.3027 - accuracy: 0.0000e+00 - val_loss: 2.2224 - val_accuracy: 0.0000e+00
Epoch 4/100
7000/7000 [==============================] - 0s 5us/sample - loss: 2.2852 - accuracy: 0.0733 - val_loss: 2.2046 - val_accuracy: 0.0000e+00
Epoch 5/100
7000/7000 [==============================] - 0s 5us/sample - loss: 2.2676 - accuracy: 0.1699 - val_loss: 2.1870 - val_accuracy: 0.0000e+00
Epoch 6/100
7000/7000 [==============================] - 0s 5us/sample - loss: 2.2498 - accuracy: 0.2306 - val_loss: 2.1694 - val_accuracy: 0.0000e+00
Epoch 7/100
7000/7000 [=========

In [20]:
#Predicting the Test set results
y_pred = model2.predict_classes(normalized_test_x)
y_pred = np.where(y_pred > .5, 1, 0)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
test_y


array([1., 1., 1., ..., 0., 0., 1.], dtype=float32)

In [22]:
#Making the COnfusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(test_y, y_pred)
cm

array([[2415,    0],
       [ 585,    0]])

In [0]:
## Model optimazation is not covered as part of residency so not included 